## Importing libraries

In [1]:
#
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

## Analysis

In [ ]:
X_train = pd.read_csv("../data/X_train.csv")
X_test = pd.read_csv("../data/X_test.csv")

In [2]:
param_grid = {'kernel':['rbf', 'sigmoid'], 'C':[0.1, 1, 10, 100, 1000, 5000], 'gamma': ['auto', 'scale']}

svr = SVR()

clf = GridSearchCV(svr, param_grid, cv = 5)
clf.fit(X_train, y_train.to_numpy().ravel())

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))
print(clf.best_params_)


NameError: name 'X_train' is not defined

In [ ]:
rfr =  GridSearchCV(RandomForestRegressor(),
                        param_grid = {'max_depth': [1, 5, 20]},
                        cv = 3,
                        scoring = 'neg_mean_squared_error')

In [ ]:
rgr = make_pipeline(preprocessor, rfr)

rgr.fit(X_train, y_train.to_numpy().ravel())


In [ ]:
# print(f"Score: {rgr.score(X_train, y_train)}")
len(X_train['property_type'].unique())

# rgr.score(X_test, y_test)

In [ ]:
X_train['property_type'].unique()

In [ ]:

X_test['property_type'].unique()

In [ ]:
# for model_name, model in rgr_models.items():
#     rgr = make_pipeline(preprocessor,
#                         model)
#     rgr.fit(X_train, y_train)
#     y_test[model_name] = rgr.predict(X_test)
#     print(f"{model_name} best hyperparams = {model.best_params_}.")
#     print(f"{model_name} RMSE = {np.sqrt(mean_squared_error(y_test[model_name], y_test['Age'])):.2f} years")